## Introducción a Pandas (2)

Durante el curso vamos a estar desarrollando los aspectos prácticos utilizando Python. Para aquellos que no estén muy familiarizados con el lenguaje, acá tenemos una introducción a las bibliotecas que más vamos a manejar.

Si necesitan más material, algunos libros para consultar (si los quieren, los podemos compartir):
* Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython
* Pandas for Everyone: Python Data Analysis

Cuentan con una serie de notebooks introductorias:
* Creación, lectura y escritura.
* **Indexando, seleccionando y asignando.**
* Tipos de datos, valores faltantes, funciones de resumen.
* Agrupamientos y orden.
* Renombrado y combinación.

En cada notebook van a encontrar algunos ejericios (opcionales).

Las notebooks se encuentran basadas en diversos tutoriales de Kaggle y cursos de la Unversidad de Berkeley.

## Indexando, seleccionando y asignando

Vamos a trabajar con el ``DataFrame`` de tweets que teníamos en la notebook anterior.

In [ ]:
import pandas as pd

In [ ]:
df_tweets = pd.read_csv('tweets.csv',index_col = 0)
df_tweets

En Python podemos acceder a las propiedades de un objeto accediéndolas como si fueran atributos. Por ejemplo, si tenemos un objeto ``libro`` que tiene un atributo ``titulo``, lo podemos acceder como ``libro.titulo``. Las columnas de un ``DataFrame`` pueden ser accedidas de la misma forma.

In [ ]:
df_tweets.user_id

También las podemos acceder como accedemos a las claves de un diccionario:

In [ ]:
# esto es equivalente
df_tweets['user_id']

### Selección basada en índices

La primera opción que tenemos para acceder a los datos es utilizar la posición numérica de los datos, es decir, su índice.

Para seleccionar la primera fila de un ``DataFrame`` podemos utilizar:

In [ ]:
df_tweets.iloc[0]

Tanto ``iloc`` como ``loc`` funcionan indicando primero la fila y luego la columna. Con lo que puede parecer que acceder a una columna sea más complicado.

Para acceder a la columna 0 usando ``iloc`` tenemos que hacer:

In [ ]:
df_tweets.iloc[:,0]

El operador ``:`` siginifica "todo". Como en este caso, seleccionamos todas las filas, y de ellas la columna 0. También se lo puede combinar con otros selectorres para indicar un rango de valores. Por ejemplo para seleccionar la columna 0 de las primeros 5 filas:

In [ ]:
df_tweets.iloc[:5,0]

O para seleccionar solo la tercera y cuarta fila:

In [ ]:
df_tweets.iloc[2:5,0]

También le podemos pasar una lista:

In [ ]:
df_tweets.iloc[[0,1,2],0]

Finalmente, también podemos utilizar números negativos. Lo que hace es contar desde el final de los valores. De esta forma, para acceder a la última fila:

In [ ]:
df_tweets.iloc[-1:]

Resumen:

```
a[start:stop]  # items start through stop-1
a[start:]      # items start through the rest of the array
a[:stop]       # items from the beginning through stop-1
a[:]           # a copy of the whole array
```

### Selección basada en etiquetas

Muy lindos los índices, pero recordemos que el ``DataFrame`` tiene nombres de columnas y filas que pueden ser utilizados para accederlos. 

Por ejemplo, para obtener la columna cero por nombre de la primera fila:
En ``loc`` por más que se le pase un ``int`` se lo interpreta como label y no como índice.

In [ ]:
df_tweets.loc[915943026534514688,'created_at']

``iloc`` es más simple que ``loc``. En ``iloc`` estamos tratando al ``DataFrame`` como si fuera una gran matrix, una lista de listas o un arreglo de arreglo como en Java, al que podemos acceder con índices. Por el contrario, ``loc`` utiliza las etiquetas. Dado que usualmente vamos a tener los ``DataFrame`` con índices significativos, suele ser más "fácil" (o intuitivo) utilizar ``loc``.

Cúal elegir? Depende de lo que se necesite o quiera. Recordar que hay una diferencia entre ambos respecto a la inclusión de los extremos:

* ``iloc``. De un rango, el primero es incluido, mientras que el último es excluido. De ``0:10``, se seleccionaran ``0...9``.
* ``loc``. De un rango, incluye ambos índices. De ``0:10``, se seleccionaran ``0...10``.

Por qué? ``loc`` puede indexar cualquier tipo, por ejemplo ``str``. En este caso, si tenemos un ``DataFrame`` con índices ``str``, por ejemplo ``Apples, ..., Potatoes...`` y queremos seleccionar todas los vegetales que estén entre Apples y Potatoes, es más conveniente escribir ``df.loc['Apples':'Potatoes']`` que hacer algo como ``df.loc['Apples':'Potatoet']`` (considerando que ``t`` está luego en el alfabeto que ``s``).

### Manipulando el índice

El índice puede ser manipulado, virtualmente, como querramos.

Por ejemplo, podemos cambiarlo usando el ``set_index``.

Nota: esta operación no se realiza por defecto sobre el ``DataFrame`` sino que retorna uno nuevo con el nuevo índice seteado. Entonces, en principio, si quisiéramos que el cambio quede en el df, deberíamos asignarlo a ``df_tweets``.

In [ ]:
df_tweets.set_index('user_id') 

Podemos acceder al index como si fuera una lista:

In [ ]:
df_tweets.index

In [ ]:
df_tweets.index[0]

### Selección condicional

Hasta ahora accedimos a los datos utilizando estructuras propias del ``DataFrame``, pero qué pasa si queremos acceder a valores que cumplen con una condición. 

Por ejemplo, queremos acceder a los tweets escritos por el usuario ``334537201``:

In [ ]:
df_tweets.user_id == 393190233

Esta operación da como resultado una serie booleana ``True/False`` basada en el ``user_id`` de cada tweet. 

Esto también puede ser utilizado dentro de un ``loc`` para seleccionar los registros relevantes:

In [ ]:
df_tweets.loc[df_tweets.user_id == 393190233]

El mismo resultado puede obtener sin usar el ``loc``.

In [ ]:
df_tweets[df_tweets['user_id'] == 393190233]

También podemos combinar las consultas. Por ejemplo, los tweets del usuario ``393190233`` que sean replies (es decir que ``replied_id > `0``)

In [ ]:
df_tweets[(df_tweets['user_id'] == 393190233) & (df_tweets['replied_id'] > 0)]

Además de los operadores lógicos "comunes", también hay otros operadores "especiales". Entre ellos:

* ``isin``. Para preguntar si un valor se encuentra dentro de los valores de una lista.
* ``isnull``, ``notnull``. Para preguntar por valores que son o no nullos o vacíos.

Para la negación se utiliza ``~`` delante. Por ejemplo: ``~df_tweets['user_id'].isin([334537201])``

In [ ]:
df_tweets[df_tweets['user_id'].isin([393190233])]

In [ ]:
df_tweets[~df_tweets['user_id'].isin([393190233])]

### Asignando valores

Se pueden asignar valores constantes o iterables. Si se asigna un valor constante, el mismo es asignado a todas las filas. Si se utiliza un iterable (por ejemplo una lista), el iterable debe tener tantos valores como elementos haya en el ``DataFrame``.

Vamos a crear una nueva columna y asignarle el mismo valor a todos los tweets:

In [ ]:
df_tweets['constante'] = 'si'
df_tweets

Ahora, creamos una columna con valores consecutivos:

In [ ]:
df_tweets['variable'] = [i for i in range(0,len(df_tweets))]
df_tweets

### Ejercicios

1. Seleccionar la columna ``created_at`` y asignarla a la variable ``creacion``.

In [ ]:
# TODO

2. Seleccionar el valor de la primera fila para la columna ``user_id`` de al menos dos formas diferentes.

In [ ]:
# TODO

In [ ]:
# TODO

3. Crear el ``DataFrame`` ``df`` conteniendo las columnas ``created_at`` y ``user_id`` para los tweets en las posiciones 0, 1, 4 y 5.

In [ ]:
# TODO

4. Crear el ``DataFrame`` ``df`` conteniendo las columnas ``created_at`` y ``user_id`` para los tweets con id impar.

In [ ]:
# TODO